

## **Environment Setup**



In [ ]:
!pip install transformers


In [ ]:
pip install sentence_transformers

In [ ]:
!pip install gensim


In [ ]:
!pip install --upgrade gensim


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk import sent_tokenize
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords

# Download stopwords list
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

from re import sub
from gensim.utils import simple_preprocess


#Similarity search for car make model

The current problem of the existing search engine is the less accurate search results from users’ inputs that might affect customer experiences. For example, when a user types ‘cc’ in the search box, the returned result is usually Honda Accord at the top, instead of Volkswagen cc.

Especially while you type the model, mostly getting the results is not accurate.

I hope AI will help us the problem

mount the drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df=pd.read_csv('/content/gdrive/MyDrive/data/car_make model_2.csv')
#df=pd.read_csv('/content/new_carmodel2 - new_carmodel2.csv')
df['CarModel']=df['CarModel'].astype(str)
df['CarModel']= df['CarModel'].apply(lambda x: x.lower())

In [ ]:
df

,CarModel
0,audi q3
1,chevrolet malibu
2,cadillac escalade esv
3,chevrolet corvette
4,acura rlx
...,...
1259,rivian r1t
1260,toyota rav4 prime
1261,volkswagen atlas cross sport
1262,gmc hummer ev


In [ ]:
df['CarModel']

0                            audi q3
1                   chevrolet malibu
2              cadillac escalade esv
3                 chevrolet corvette
4                          acura rlx
                    ...             
1259                      rivian r1t
1260               toyota rav4 prime
1261    volkswagen atlas cross sport
1262                   gmc hummer ev
1263             jeep grand wagoneer
Name: CarModel, Length: 1264, dtype: object

In [ ]:
df['CarModel'].shape

(1264,)

In [ ]:
corpus = df['CarModel'].astype(str).values.tolist()


In [ ]:
#put the search terms, and corpus
base_document = corpus
#query = 'cc'
query = 'bmw'

#threhold
highest_score = 1
lowest_score = 0.55

#top result
top_k = 5


In [ ]:
import operator
from itertools import islice


def get_best_similar(best_suit):
  sorted_d = dict( sorted(best_suit.items(), key=operator.itemgetter(1),reverse=True))
  return (dict(islice(sorted_d.items(), top_k)))



## **BERT**

We came out with solution to solve this issue is using BERT pre-train model.

What is BERT?

**BERT** (introduced in [this paper](https://arxiv.org/abs/1810.04805)) stands for Bidirectional Encoder Representations from Transformers. It is a trained Transformer Encoder stack. BERT is a model that broke several records for how well models can handle language-based tasks after the release paper. It is saving the time, energy, knowledge, and resources that would have gone to training a language-processing model from scratch. Its the good point.

The paper presents two model sizes for BERT:
BERT BASE (L=12, H=768, A=12, Total Parameters=110M) and BERT LARGE (L=24, H=1024, A=16, Total Parameters=340M).In this project, i will use bert-base.


more clear explaination in [it](https://jalammar.github.io/illustrated-bert/)

In [ ]:
#def cosine_similarities_test(query_embedding, corpus_embeddings, top_k=5):
#	cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
#	top_results = torch.topk(cos_scores, k=top_k)
#	for score, idx in zip(top_results[0], top_results[1]):
#		print(corpus[idx], "(Score: {:.4f})".format(score))


In [ ]:
def get_bert_similarity(query, base_document):
	# This will download and load the pretrained model offered by UKPLab.
	model = SentenceTransformer('all-MiniLM-L6-v2')

	# Although it is not explicitly stated in the official document of sentence transformer, the original BERT is meant for a shorter sentence.
	#We will feed the model by sentences instead of the whole documents.
	base_embeddings_sentences = model.encode(base_document)
	for document in [query]:
		embeddings_sentences = model.encode(document)
		cos_scores = util.pytorch_cos_sim(embeddings_sentences, base_embeddings_sentences)[0]
		top_results = torch.topk(cos_scores, k=top_k)
		print("\n\n======================\n\n")
		print("Query:", document)
		print("\nTop 5 most similar sentences in corpus:")
		for score, idx in zip(top_results[0], top_results[1]):
			if score >= 0.5:
				print(corpus[idx], "(Score: {:.4f})".format(score))
			#if lowest_score <= score and highest_score >= score:
			#	most_similar_document = corpus[idx]
				#print(corpus[idx], "(Score: {:.4f})".format(score))

		#cosine_similarities_test(embeddings_sentences, base_embeddings_sentences, top_k=5)
		#hits = util.semantic_search(embeddings_sentences, base_embeddings_sentences, top_k=5)
		#print(hits)
		#hits = hits[0]      #Get the hits for the first query
		#for hit in hits:
		#	print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))

print("BERT approach:")
get_bert_similarity(query, base_document)

BERT approach:




Query: bmw

Top 5 most similar sentences in corpus:
bmw m3 (Score: 0.8521)
bmw m (Score: 0.8281)
bmw i8 (Score: 0.8146)
bmw z8 (Score: 0.8105)
bmw z3 (Score: 0.8067)


## **TF-idf (stands for term frequency-inverse document frequency)**

I have performed another approach which is TF-idf

what is [TF-idf](http://www.tfidf.com/) ?

TF: Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization:

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).

IDF: Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following:

IDF(t) = log_e(Total number of documents / Number of documents with term t in it).

Example:

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.





In [ ]:
def get_simple_Tfidf_similarity(search_terms, documents):
	best_suit = {}
	doc_vectors = TfidfVectorizer().fit_transform([search_terms] + documents)

	cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
	document_scores = [item.item() for item in cosine_similarities[1:]]

	highest_score_index = 0
	#count = 0
	for i, score in enumerate(document_scores):
		#if count == 5:
		#	break

		if lowest_score <= score and highest_score >= score:
			#count = count +1
			highest_score_index = i
			most_similar_document = documents[highest_score_index]
			best_suit[most_similar_document] = 	score
			#print("Most similar document by TF-IDF with the score: ",most_similar_document, " (Score: {:.4f})".format(score))
	return best_suit

print("TF-idf approach:")

print("\n\n======================\n\n")
print("Query:", query)

print("\nTop 5 most similar sentences in corpus:")

best_suit = get_simple_Tfidf_similarity(query, base_document)

for key, value in get_best_similar(best_suit).items():
  print("Most similar document by TF-IDF with the score: ",key, " (Score: {:.4f})".format(value))



TF-idf approach:




Query: bmw

Top 5 most similar sentences in corpus:
Most similar document by TF-IDF with the score:  bmw m  (Score: 1.0000)
Most similar document by TF-IDF with the score:  bmw 3 series  (Score: 0.6760)
Most similar document by TF-IDF with the score:  bmw 7 series  (Score: 0.6760)
Most similar document by TF-IDF with the score:  bmw 4 series  (Score: 0.6760)
Most similar document by TF-IDF with the score:  bmw 2 series  (Score: 0.6760)


by lemmatizer approach

In [ ]:
stop_words = set(stopwords.words('english'))

# Interface lemma tokenizer from nltk with sklearn
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

# Lemmatize the stop words
tokenizer=LemmaTokenizer()
token_stop = tokenizer(' '.join(stop_words))




def get_lemma_Tfidf_similarity(search_terms, documents):
	best_suit = {}

	# Create TF-idf model
	vectorizer = TfidfVectorizer(stop_words=token_stop,tokenizer=tokenizer)

	doc_vectors = vectorizer.fit_transform([search_terms] + documents)

	# Calculate similarity
	cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
	document_scores = [item.item() for item in cosine_similarities[1:]]

	#print(document_scores)
	highest_score_index = 0
	for i, score in enumerate(document_scores):
		if lowest_score <= score and highest_score >= score:
			highest_score_index = i
			most_similar_document = documents[highest_score_index]
			best_suit[most_similar_document] = 	score
			#print("Most similar document by TF-IDF with the score: ",most_similar_document, " (Score: {:.4f})".format(score))
	return best_suit

print("TF-idf approach:")

print("\n\n======================\n\n")
print("Query:", query)

print("\nTop 5 most similar sentences in corpus:")

best_suit = get_lemma_Tfidf_similarity(query, base_document)

for key, value in get_best_similar(best_suit).items():
  print("Most similar document by TF-IDF with the score: ",key, " (Score: {:.4f})".format(value))


TF-idf approach:




Query: bmw

Top 5 most similar sentences in corpus:
Most similar document by TF-IDF with the score:  bmw m  (Score: 1.0000)
Most similar document by TF-IDF with the score:  bmw x5  (Score: 0.5532)
Most similar document by TF-IDF with the score:  bmw x6 m  (Score: 0.5532)
Most similar document by TF-IDF with the score:  bmw x6  (Score: 0.5532)
Most similar document by TF-IDF with the score:  bmw z4  (Score: 0.5532)



## **GloVe**


GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.

it maps words into numerical vectors — points in a multi-dimensional space so that words that occur together often are near each other in space. It is an unsupervised learning algorithm, developed at Stanford University in 2014. [more details](https://nlp.stanford.edu/projects/glove/)

In [ ]:

stopwords = ['the', 'and', 'are', 'a']

def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

def get_glove_similarity(query_string, documents):
	best_suit = {}
	# Preprocess the documents, including the query string
	corpus = [preprocess(document) for document in documents]
	query = preprocess(query_string)

	glove = api.load("glove-wiki-gigaword-50")
	similarity_index = WordEmbeddingSimilarityIndex(glove)

	# Build the term dictionary, TF-idf model
	dictionary = Dictionary(corpus+[query])
	tfidf = TfidfModel(dictionary=dictionary)

	# Create the term similarity matrix.
	similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

	# Compute Soft Cosine Measure between the query and the documents.
	query_tf = tfidf[dictionary.doc2bow(query)]

	index = SoftCosineSimilarity(
				tfidf[[dictionary.doc2bow(document) for document in corpus]],
				similarity_matrix)

	doc_similarity_scores = index[query_tf]
	print(doc_similarity_scores)
	# Output the sorted similarity scores and documents
	sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
	for idx in sorted_indexes:
		if lowest_score <= doc_similarity_scores[idx] and highest_score >= doc_similarity_scores[idx]:
			highest_score_index = idx
			most_similar_document = documents[highest_score_index]
			best_suit[most_similar_document] = 	doc_similarity_scores[idx]

	return best_suit

best_suit = get_glove_similarity(query, base_document)
#print(sorted_indexes)

for key, value in get_best_similar(best_suit).items():
  print("Most similar document by golve with the score: ",key, " (Score: {:.4f})".format(value))

100%|██████████| 752/752 [00:15<00:00, 48.56it/s]
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


[0.69495046 0.39313257 0.         ... 0.69939244 0.         0.        ]
Most similar document by golve with the score:  bmw m  (Score: 1.0000)
Most similar document by golve with the score:  bmw x3  (Score: 1.0000)
Most similar document by golve with the score:  bmw 7 series  (Score: 1.0000)
Most similar document by golve with the score:  bmw m5  (Score: 1.0000)
Most similar document by golve with the score:  bmw 2 series  (Score: 1.0000)
